In [1]:
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.rpm.sh | sudo bash
!sudo yum install git-lfs -y
!git lfs install

Detected operating system as amzn/2.
Checking for curl...
Detected curl...
done.
Installing pygpgme to verify GPG signatures...
Loaded plugins: dkms-build-requires, extras_suggestions, langpacks, priorities,
              : update-motd, versionlock
amzn2-core                                               | 3.7 kB     00:00     
github_git-lfs-source/signature                          |  833 B     00:00     
Retrieving key from https://packagecloud.io/github/git-lfs/gpgkey
Importing GPG key 0xDC282033:
 Userid     : "https://packagecloud.io/github/git-lfs (https://packagecloud.io/docs#gpg_signing) <support@packagecloud.io>"
 Fingerprint: 6d39 8dbd 30dd 7894 1e2c 4797 fe2a 5f8b dc28 2033
 From       : https://packagecloud.io/github/git-lfs/gpgkey
github_git-lfs-source/signature                          | 1.8 kB     00:00 !!! 
neuron                                                   | 2.9 kB     00:00     
github_git-lfs-source/primary                              | 4.8 kB   00:00     
gi

In [2]:
# Define the my final model name
mymodel_name='RuPERTa_base_sentiment_analysis_es'
local_path = mymodel_name

In [4]:
import sagemaker
print(sagemaker.__version__)

# Specify your bucket name
bucket_name = 'edumunozsala-ml-sagemaker'

sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
# Specify your bucket name
sagemaker_session_bucket= bucket_name
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

role = sagemaker.get_execution_role()
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)
region = sess.boto_session.region_name

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

2.68.0
sagemaker role arn: arn:aws:iam::223817798831:role/workshop-sagemaker-kfp-role
sagemaker bucket: edumunozsala-ml-sagemaker
sagemaker session region: us-east-1


In [14]:
import os
import tarfile
from sagemaker.s3 import S3Downloader

local_path = mymodel_name
model_data= 's3://edumunozsala-ml-sagemaker/ruperta-sentiment/ruperta-sentiment-analysis-single-gpu-2021-12-05-18-51-02/output/'

os.makedirs(local_path, exist_ok = True)

In [7]:
from getpass import getpass

hf_username = "edumunozsala" # your username on huggingface.co
hf_email = "edumunozsala@gmail.com" # email used for commit
repository_name = mymodel_name
password = getpass("Enter your password:") # creates a prompt for entering password

Enter your password:········


In [15]:
repository_name,  local_path

('RuPERTa_base_sentiment_analysis_es', 'RuPERTa_base_sentiment_analysis_es')

In [9]:
from huggingface_hub import HfApi, Repository

# get hf token
token = HfApi().login(username=hf_username, password=password)

ERROR:root:HfApi.login: This method is deprecated in favor of `set_access_token`.


In [10]:
token

'fyBjORyHzEbmVyQEcZbnTFQprgqtKaYaqIlRdflgjbFMyNgcjLRUfwVbstmoheLJOErmSOdGDlmOWUjNadXfHmizpdKpYVASPSWvJKthJdoCjUxOQqiwHMMcCvzgJscj'

In [16]:
# create repository
repo_url = HfApi().create_repo(token=token, name=repository_name, exist_ok=True)

In [17]:
repo_url

'https://huggingface.co/edumunozsala/RuPERTa_base_sentiment_analysis_es'

In [18]:
# create a Repository instance
model_repo = Repository(use_auth_token=token,
                        clone_from=repo_url,
                        local_dir=local_path,
                        git_user=hf_username,
                        git_email=hf_email)

Cloning https://huggingface.co/edumunozsala/RuPERTa_base_sentiment_analysis_es into local empty directory.


In [19]:
# download model from S3
S3Downloader.download(
    s3_uri=model_data, # s3 uri where the trained model is located
    local_path=local_path, # local path where *.tar.gz will be saved
    sagemaker_session=sess # sagemaker session used for training the model
)

# unzip model
tar = tarfile.open(f"{local_path}/model.tar.gz", "r:gz")
tar.extractall(path=local_path)
tar.close()
os.remove(f"{local_path}/model.tar.gz")

In [20]:
# push model to the hub
model_repo.push_to_hub()

Upload file checkpoint-1329/optimizer.pt:   0%|          | 32.0k/961M [00:00<?, ?B/s]

Upload file checkpoint-1329/scaler.pt: 100%|##########| 559/559 [00:00<?, ?B/s]

Upload file checkpoint-1329/pytorch_model.bin:   0%|          | 32.0k/481M [00:00<?, ?B/s]

Upload file checkpoint-1329/rng_state.pth: 100%|##########| 14.3k/14.3k [00:00<?, ?B/s]

Upload file checkpoint-1329/training_args.bin: 100%|##########| 2.36k/2.36k [00:00<?, ?B/s]

Upload file checkpoint-1329/scheduler.pt: 100%|##########| 623/623 [00:00<?, ?B/s]

Upload file output.tar.gz: 100%|##########| 302/302 [00:00<?, ?B/s]

To https://huggingface.co/edumunozsala/RuPERTa_base_sentiment_analysis_es
   97737b2..0d59578  main -> main

   97737b2..0d59578  main -> main



'https://huggingface.co/edumunozsala/RuPERTa_base_sentiment_analysis_es/commit/0d59578afe01a669da4e1af0bcd7022a00acf865'